In [233]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [234]:
n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 112

In [235]:
def logits_function(p, X):
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [236]:
# Forward propagation
def forward_prop(params, X,y):
    logits = logits_function(params, X)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [237]:
def Get_PSO(f):
    # Initialize swarm
    options = {'c1': 2.05, 'c2': 2.05, 'w': 0.3}

    # Call instance of PSO
    dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
    optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=dimensions, options=options)

    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=1000)
    return pos

In [238]:
def predict(pos, X):
    logits = logits_function(pos, X)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [239]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")


labelencoder = LabelEncoder()

def Start_Iris():
    data = pd.read_csv('databases/iris.data', names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
    target = labelencoder.fit_transform(data['class'].values)
    data = data.drop('class', axis = 1).values

    sc = StandardScaler()
    data = sc.fit_transform(data)

    x_train, x_test, y_train, y_test = train_test_split(data, target)

    def f(x):
        n_particles = x.shape[0]
        j = [forward_prop(x[i], x_train, y_train) for i in range(n_particles)]
        return np.array(j)

    pos = Get_PSO(f)
    scores = (predict(pos, x_test) == y_test)

    print("\n Iris \n")
    print("Accuracy: %.2f%%" % (100 * np.mean(scores)))

In [240]:
Start_Iris()

2022-03-25 13:39:03,647 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 2.05, 'c2': 2.05, 'w': 0.3}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=0.00298
2022-03-25 13:39:06,837 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0029848305292933357, best pos: [-1.18307257e+00  8.08837216e+00  7.34947393e-01  8.54152247e-03
 -1.82157458e+00  3.87804614e+00  8.43515572e-01  2.49026358e+00
  2.55805664e-01  1.79553664e+00 -9.17922829e+00  7.07700472e-01
  1.99477885e+00  2.92821390e-01  1.16680272e+00 -9.35953145e-01
  2.75307402e+00 -1.26321580e+00  8.57603312e-01  6.17356555e-01
  2.75251983e-01  6.62744678e-02 -4.98173609e-01  3.41492146e+00
 -3.51253661e-01  1.92662285e-02 -2.30342416e-01  1.57220346e+01
  2.34787057e+01  1.17896281e+00  2.16996255e+00 -6.69041231e-01
  1.96616249e+00  1.07137949e+00 -2.94176776e+00  2.10688891e-01
  9.83225678e-01 -8.04537353e-01  4.09876173e-01  1.50855695e+00
  6.12373008e-01 -


 Iris 

Accuracy: 97.37%
